# DSMLC Annual Final Competition

## Imports

In [38]:
!pip install lightgbm
!pip install catboost
!pip install xgboost

import math
import numpy as np
import pandas as pd
import csv
import sys

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

from lightgbm import LGBMRegressor

from xgboost.sklearn import XGBRegressor

from catboost import CatBoostRegressor

from sklearn.linear_model import SGDRegressor

from sklearn.kernel_ridge import KernelRidge

from sklearn.linear_model import ElasticNet

from sklearn.linear_model import BayesianRidge

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.svm import SVR

import time
import random
import requests
from datetime import datetime


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Data

In [2]:
def load_data():
    # The columns we will make
    column_names = ['Year', 'Country', 'Happiness', 'Economy', 'Health', 'Freedom', 'Generosity', 'Corruption']
    main_dataframe = pd.DataFrame(columns=column_names)
    
    # Import xls sheet
    xls = pd.ExcelFile('World Happiness Datasets (2015-2022).xlsx')
    
    for i in range(2015, 2023):
        # Read Dataframe In
        dataframe = pd.read_excel(xls, f'{i}')
        
        # Read in year
        year = [i for _ in dataframe[dataframe.columns[0]]]
        dataframe['Year'] = year
        
        # Get Old Columns
        old_columns = []
        if i == 2015 or i == 2016:
            old_columns = ['Year', 'Country','Happiness Score', 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom', 'Generosity', 'Trust (Government Corruption)']
        elif i == 2017:
            old_columns = ['Year', 'Country','Happiness.Score', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.']
        elif i == 2018 or i == 2019:
            old_columns = ['Year', 'Country or region','Score', 'GDP per capita', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
        elif i == 2020 or i == 2021:
            old_columns = ['Year', 'Country name','Ladder score', 'Explained by: Log GDP per capita', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption']
        else:
            old_columns = ['Year', 'Country','Happiness score', 'Explained by: GDP per capita', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption']
        
        # Get Only Needed Columns
        dataframe = dataframe.filter(old_columns)
        
        # Rename Columns
        for column in range(len(column_names)):
            dataframe = dataframe.rename(columns={
                old_columns[column]: column_names[column]
            })
        
        # Modify needed columns
        if i == 2018:
            dataframe['Economy'] = dataframe['Economy'].apply(lambda x: x/1000)
            dataframe['Happiness'] = dataframe['Happiness'].apply(lambda x: x/1000)
        np.seterr(divide = 'ignore') 
        if i != 2020 and i != 2021:
            dataframe['Economy'] = np.log10(dataframe['Economy'])
        np.seterr(divide = 'warn')
        
        
        # Add dataframe to main dataframe
        main_dataframe = pd.concat([main_dataframe, dataframe])
    
    # Replace inf and -inf with NaN, then Imputer all NaN's
    main_dataframe = main_dataframe.replace([np.inf, -np.inf], np.NaN)
    main_dataframe = main_dataframe.fillna(main_dataframe.mean(numeric_only=True))
    
    return main_dataframe
    
    

In [3]:
all_data = load_data()

## Create Test and Training Sets

In [4]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [5]:
train_set, test_set = split_train_test(all_data, 0.2)

In [6]:
len(train_set)

985

In [7]:
len(test_set)

246

## Transformation Pipeline

In [8]:
def transform_data(data):
    
    # One Hot Encode 'Country' column
    one_hot_encoder = OneHotEncoder()
    data_one_hot = one_hot_encoder.fit_transform(data['Country'].values.reshape(-1,1))
    one_hot_categories = one_hot_encoder.categories_
    one_hot_data = data_one_hot.toarray()
    one_hot_dataframe = pd.DataFrame(
        data=one_hot_data,
        columns=one_hot_categories)
    
    # Normalize every column
    numberic_data = data.drop(columns=['Country'])
    for column in numberic_data:
        numberic_data[column] = (numberic_data[column] - numberic_data[column].min()) / (numberic_data[column].max() - numberic_data[column].min()) 
    
    # Get two data frames
    data_prepped = pd.concat([numberic_data.reset_index(drop=True), one_hot_dataframe.reset_index(drop=True)], axis=1)
    data_y = data_prepped.filter(['Generosity'])
    data_x = data_prepped.drop(columns=['Generosity'])
    
    return data_x, data_y

In [9]:
train_set_x, train_set_y = transform_data(train_set)

In [10]:
train_set_x.describe

<bound method NDFrame.describe of          Year     Happiness   Economy    Health   Freedom  Corruption  \
0    0.285714  8.406978e-04  0.587857  0.622243  0.557743    0.226191   
1    0.714286  6.239351e-04  0.693929  0.741744  0.624251    0.043205   
2    0.714286  5.350079e-04  0.613751  0.300378  0.410188    0.195616   
3    0.571429  6.033879e-04  0.577859  0.411043  0.525676    0.093697   
4    0.142857  5.412475e-04  0.565909  0.247257  0.512676    0.166150   
..        ...           ...       ...       ...       ...         ...   
980  0.857143  6.483949e-04  0.600427  0.146363  0.697297    0.267462   
981  0.142857  5.697605e-04  0.581387  0.163111  0.574095    0.143356   
982  0.000000  8.277838e-04  0.575456  0.605408  0.722514    0.087223   
983  0.857143  7.846947e-04  0.740181  0.521472  0.762162    0.001704   
984  0.428571  1.137962e-07  0.186227  0.235758  0.000000    0.103918   

     (Afghanistan,)  (Albania,)  (Algeria,)  (Angola,)  ...  (United States,)  \
0       

In [11]:
for col in train_set_y:
    print(col)

Generosity


## Train Model

In [12]:
models = []

### Linear Regression

In [13]:
lin_reg = LinearRegression()
lin_reg.fit(train_set_x.values, train_set_y.values)

models.append({
    'model_name': 'Standrd Linear Regression',
    'model': lin_reg
})

### Decision Trees

In [14]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_set_x.values, train_set_y.values)

models.append({
    'model_name': 'Decision Tree Regressor',
    'model': tree_reg
})

### Random Forest Regressor

In [15]:
forest_reg = RandomForestRegressor()
forest_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Random Forest Regressor',
    'model': forest_reg
})

In [16]:
lgbm_reg = LGBMRegressor()
lgbm_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'LGBM Regressor',
    'model': lgbm_reg
})

In [17]:
xgb_reg = XGBRegressor()
xgb_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'XGBoost Regressor',
    'model': xgb_reg
})

In [18]:
cat_reg = CatBoostRegressor()
cat_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'CatBoost Regressor',
    'model': cat_reg
})

Learning rate set to 0.040845
0:	learn: 0.1343974	total: 61.5ms	remaining: 1m 1s
1:	learn: 0.1329944	total: 63.3ms	remaining: 31.6s
2:	learn: 0.1317481	total: 64.9ms	remaining: 21.6s
3:	learn: 0.1307535	total: 66.3ms	remaining: 16.5s
4:	learn: 0.1297308	total: 67.9ms	remaining: 13.5s
5:	learn: 0.1286614	total: 69.5ms	remaining: 11.5s
6:	learn: 0.1276430	total: 70.9ms	remaining: 10.1s
7:	learn: 0.1267200	total: 72.5ms	remaining: 8.98s
8:	learn: 0.1257399	total: 74.1ms	remaining: 8.16s
9:	learn: 0.1248524	total: 75.8ms	remaining: 7.51s
10:	learn: 0.1241744	total: 77.4ms	remaining: 6.96s
11:	learn: 0.1232503	total: 78.9ms	remaining: 6.5s
12:	learn: 0.1223605	total: 80.4ms	remaining: 6.1s
13:	learn: 0.1216730	total: 81.9ms	remaining: 5.77s
14:	learn: 0.1208782	total: 83.5ms	remaining: 5.48s
15:	learn: 0.1201600	total: 85ms	remaining: 5.22s
16:	learn: 0.1194253	total: 86.5ms	remaining: 5s
17:	learn: 0.1187378	total: 88ms	remaining: 4.8s
18:	learn: 0.1181383	total: 89.1ms	remaining: 4.6s
19:

196:	learn: 0.0801913	total: 383ms	remaining: 1.56s
197:	learn: 0.0800467	total: 385ms	remaining: 1.56s
198:	learn: 0.0798924	total: 387ms	remaining: 1.56s
199:	learn: 0.0797496	total: 389ms	remaining: 1.55s
200:	learn: 0.0796116	total: 391ms	remaining: 1.55s
201:	learn: 0.0794555	total: 393ms	remaining: 1.55s
202:	learn: 0.0793088	total: 394ms	remaining: 1.55s
203:	learn: 0.0791581	total: 396ms	remaining: 1.54s
204:	learn: 0.0790010	total: 398ms	remaining: 1.54s
205:	learn: 0.0789231	total: 400ms	remaining: 1.54s
206:	learn: 0.0787764	total: 402ms	remaining: 1.54s
207:	learn: 0.0786687	total: 404ms	remaining: 1.54s
208:	learn: 0.0785310	total: 407ms	remaining: 1.54s
209:	learn: 0.0784019	total: 408ms	remaining: 1.54s
210:	learn: 0.0782529	total: 410ms	remaining: 1.53s
211:	learn: 0.0781012	total: 412ms	remaining: 1.53s
212:	learn: 0.0779850	total: 414ms	remaining: 1.53s
213:	learn: 0.0778699	total: 415ms	remaining: 1.52s
214:	learn: 0.0777782	total: 417ms	remaining: 1.52s
215:	learn: 

422:	learn: 0.0614450	total: 750ms	remaining: 1.02s
423:	learn: 0.0613844	total: 752ms	remaining: 1.02s
424:	learn: 0.0613179	total: 754ms	remaining: 1.02s
425:	learn: 0.0612534	total: 756ms	remaining: 1.02s
426:	learn: 0.0612001	total: 758ms	remaining: 1.02s
427:	learn: 0.0611599	total: 759ms	remaining: 1.01s
428:	learn: 0.0610890	total: 761ms	remaining: 1.01s
429:	learn: 0.0609595	total: 763ms	remaining: 1.01s
430:	learn: 0.0609312	total: 765ms	remaining: 1.01s
431:	learn: 0.0608642	total: 768ms	remaining: 1.01s
432:	learn: 0.0607973	total: 769ms	remaining: 1.01s
433:	learn: 0.0607701	total: 771ms	remaining: 1s
434:	learn: 0.0607113	total: 772ms	remaining: 1s
435:	learn: 0.0606683	total: 774ms	remaining: 1s
436:	learn: 0.0605977	total: 775ms	remaining: 999ms
437:	learn: 0.0605528	total: 777ms	remaining: 996ms
438:	learn: 0.0605325	total: 778ms	remaining: 994ms
439:	learn: 0.0604760	total: 780ms	remaining: 992ms
440:	learn: 0.0604173	total: 781ms	remaining: 990ms
441:	learn: 0.0603974

642:	learn: 0.0505656	total: 1.13s	remaining: 630ms
643:	learn: 0.0505503	total: 1.14s	remaining: 628ms
644:	learn: 0.0505085	total: 1.14s	remaining: 626ms
645:	learn: 0.0503971	total: 1.14s	remaining: 624ms
646:	learn: 0.0503568	total: 1.14s	remaining: 622ms
647:	learn: 0.0503260	total: 1.14s	remaining: 621ms
648:	learn: 0.0502976	total: 1.14s	remaining: 619ms
649:	learn: 0.0502826	total: 1.15s	remaining: 617ms
650:	learn: 0.0502349	total: 1.15s	remaining: 616ms
651:	learn: 0.0501961	total: 1.15s	remaining: 614ms
652:	learn: 0.0501614	total: 1.15s	remaining: 612ms
653:	learn: 0.0501246	total: 1.15s	remaining: 610ms
654:	learn: 0.0501098	total: 1.15s	remaining: 608ms
655:	learn: 0.0500768	total: 1.16s	remaining: 606ms
656:	learn: 0.0500442	total: 1.16s	remaining: 604ms
657:	learn: 0.0500098	total: 1.16s	remaining: 603ms
658:	learn: 0.0499651	total: 1.16s	remaining: 601ms
659:	learn: 0.0499200	total: 1.16s	remaining: 599ms
660:	learn: 0.0498207	total: 1.16s	remaining: 597ms
661:	learn: 

886:	learn: 0.0424586	total: 1.52s	remaining: 194ms
887:	learn: 0.0424348	total: 1.52s	remaining: 192ms
888:	learn: 0.0424145	total: 1.52s	remaining: 190ms
889:	learn: 0.0423962	total: 1.52s	remaining: 189ms
890:	learn: 0.0423845	total: 1.53s	remaining: 187ms
891:	learn: 0.0423473	total: 1.53s	remaining: 185ms
892:	learn: 0.0423268	total: 1.53s	remaining: 183ms
893:	learn: 0.0423110	total: 1.53s	remaining: 182ms
894:	learn: 0.0422931	total: 1.53s	remaining: 180ms
895:	learn: 0.0422702	total: 1.54s	remaining: 178ms
896:	learn: 0.0422435	total: 1.54s	remaining: 177ms
897:	learn: 0.0422252	total: 1.54s	remaining: 175ms
898:	learn: 0.0422024	total: 1.54s	remaining: 173ms
899:	learn: 0.0421851	total: 1.54s	remaining: 172ms
900:	learn: 0.0421655	total: 1.55s	remaining: 170ms
901:	learn: 0.0421422	total: 1.55s	remaining: 168ms
902:	learn: 0.0421153	total: 1.55s	remaining: 167ms
903:	learn: 0.0420985	total: 1.55s	remaining: 165ms
904:	learn: 0.0420870	total: 1.56s	remaining: 163ms
905:	learn: 

In [19]:
sgd_reg = SGDRegressor()
sgd_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Stochastic Gradient Descent Regression',
    'model': sgd_reg
})

In [20]:
kernal_reg = KernelRidge()
kernal_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Kernel Ridge Regression',
    'model': kernal_reg
})

In [21]:
elastic_reg = ElasticNet()
elastic_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Elastic Net Regression',
    'model': elastic_reg
})

In [22]:
ridge_reg = BayesianRidge()
ridge_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Bayesian Ridge Regression',
    'model': ridge_reg
})

In [23]:
gboost_reg = GradientBoostingRegressor()
gboost_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Gradient Boosting Regression',
    'model': gboost_reg
})

In [24]:
sv_reg = SVR()
sv_reg.fit(train_set_x.values, train_set_y.values.ravel())

models.append({
    'model_name': 'Support Vector Machine',
    'model': sv_reg
})

## Evaluate Model

In [25]:
def evaluate_model(model_name, model):
    scores = cross_val_score(model, train_set_x.values, train_set_y.values.ravel(),
                            scoring="neg_mean_squared_error", cv=10)
    
    rmse = np.sqrt(-scores)

    print(f'###\t{model_name.upper()}\t###')
    print(f'\nMean:\t\t{scores.mean()}')
    print(f'Standard Dev:\t{scores.std()}')
    print(f'Scores:')
    for score in scores:
        print(f'\t{score}')
    print(f'\nNon-Normalized\nError Range:\t\t{rmse.mean()}\nMin Value In Dataset:\t{min(train_set_y.values)[0]}\nMax Value In Dataset:\t{max(train_set_y.values)[0]}')
    print(f'\nNormalized\nError Range:\t\t{(rmse.mean()-min(train_set_y.values)[0])/(max(train_set_y.values)[0] - min(train_set_y.values)[0])}\nMin Value In Dataset:\t{(min(train_set_y.values)[0]-min(train_set_y.values)[0])/(max(train_set_y.values)[0] - min(train_set_y.values)[0])}\nMax Value In Dataset:\t{(max(train_set_y.values)[0]-min(train_set_y.values)[0])/(max(train_set_y.values)[0] - min(train_set_y.values)[0])}')
    print('\n\n')
    
    return (rmse.mean()-min(train_set_y.values)[0])/(max(train_set_y.values)[0] - min(train_set_y.values)[0])

In [26]:
results = []

for model in models:
    results.append({
        'model': model['model'],
        'model_name': model['model_name'],
        'result': evaluate_model(model['model_name'], model['model'])
    })

###	STANDRD LINEAR REGRESSION	###

Mean:		-2.0869855043556355e+23
Standard Dev:	4.0670063139598296e+23
Scores:
	-1.3501735708641458e+24
	-4.2551755764353125e+23
	-2.916436786428558e+18
	-3.7709317250384763e+20
	-4.7360685904811164e+20
	-8.790830050365394e+20
	-9.786216512186637e+20
	-4.788703714218448e+20
	-2.9208216403008192e+23
	-1.6022020321861238e+22

Non-Normalized
Error Range:		260701567128.86816
Min Value In Dataset:	0.0
Max Value In Dataset:	1.0

Normalized
Error Range:		260701567128.86816
Min Value In Dataset:	0.0
Max Value In Dataset:	1.0



###	DECISION TREE REGRESSOR	###

Mean:		-0.010655603548282242
Standard Dev:	0.002217195644612794
Scores:
	-0.010505971566097762
	-0.01013611421786385
	-0.015339445662666347
	-0.007582013140426403
	-0.010804509665042643
	-0.011120286228605638
	-0.007979324868560282
	-0.009516786518478869
	-0.013550706195833916
	-0.010020877419246711

Non-Normalized
Error Range:		0.10268946140770178
Min Value In Dataset:	0.0
Max Value In Dataset:	1.0

Norma

129:	learn: 0.0888777	total: 242ms	remaining: 1.62s
130:	learn: 0.0887610	total: 244ms	remaining: 1.62s
131:	learn: 0.0886301	total: 246ms	remaining: 1.62s
132:	learn: 0.0884926	total: 248ms	remaining: 1.61s
133:	learn: 0.0883597	total: 249ms	remaining: 1.61s
134:	learn: 0.0882700	total: 251ms	remaining: 1.61s
135:	learn: 0.0881348	total: 253ms	remaining: 1.6s
136:	learn: 0.0879962	total: 255ms	remaining: 1.6s
137:	learn: 0.0878755	total: 258ms	remaining: 1.61s
138:	learn: 0.0876971	total: 259ms	remaining: 1.6s
139:	learn: 0.0876016	total: 261ms	remaining: 1.6s
140:	learn: 0.0874801	total: 262ms	remaining: 1.6s
141:	learn: 0.0873755	total: 264ms	remaining: 1.59s
142:	learn: 0.0872562	total: 265ms	remaining: 1.59s
143:	learn: 0.0871480	total: 267ms	remaining: 1.59s
144:	learn: 0.0870506	total: 269ms	remaining: 1.58s
145:	learn: 0.0869119	total: 270ms	remaining: 1.58s
146:	learn: 0.0867752	total: 272ms	remaining: 1.58s
147:	learn: 0.0866296	total: 273ms	remaining: 1.57s
148:	learn: 0.086

357:	learn: 0.0653463	total: 625ms	remaining: 1.12s
358:	learn: 0.0652862	total: 628ms	remaining: 1.12s
359:	learn: 0.0652092	total: 630ms	remaining: 1.12s
360:	learn: 0.0651853	total: 631ms	remaining: 1.12s
361:	learn: 0.0651445	total: 633ms	remaining: 1.12s
362:	learn: 0.0650624	total: 635ms	remaining: 1.11s
363:	learn: 0.0649870	total: 637ms	remaining: 1.11s
364:	learn: 0.0648991	total: 639ms	remaining: 1.11s
365:	learn: 0.0648759	total: 641ms	remaining: 1.11s
366:	learn: 0.0647967	total: 643ms	remaining: 1.11s
367:	learn: 0.0647214	total: 645ms	remaining: 1.11s
368:	learn: 0.0646368	total: 646ms	remaining: 1.1s
369:	learn: 0.0645771	total: 648ms	remaining: 1.1s
370:	learn: 0.0645382	total: 650ms	remaining: 1.1s
371:	learn: 0.0645118	total: 652ms	remaining: 1.1s
372:	learn: 0.0644371	total: 654ms	remaining: 1.1s
373:	learn: 0.0643651	total: 656ms	remaining: 1.1s
374:	learn: 0.0642788	total: 657ms	remaining: 1.09s
375:	learn: 0.0641983	total: 659ms	remaining: 1.09s
376:	learn: 0.0641

605:	learn: 0.0523612	total: 1.01s	remaining: 657ms
606:	learn: 0.0523216	total: 1.01s	remaining: 655ms
607:	learn: 0.0522930	total: 1.01s	remaining: 654ms
608:	learn: 0.0522581	total: 1.02s	remaining: 653ms
609:	learn: 0.0522184	total: 1.02s	remaining: 651ms
610:	learn: 0.0521700	total: 1.02s	remaining: 649ms
611:	learn: 0.0521325	total: 1.02s	remaining: 648ms
612:	learn: 0.0520918	total: 1.02s	remaining: 646ms
613:	learn: 0.0520196	total: 1.02s	remaining: 645ms
614:	learn: 0.0519823	total: 1.03s	remaining: 643ms
615:	learn: 0.0519484	total: 1.03s	remaining: 642ms
616:	learn: 0.0519181	total: 1.03s	remaining: 641ms
617:	learn: 0.0518688	total: 1.03s	remaining: 639ms
618:	learn: 0.0518354	total: 1.03s	remaining: 637ms
619:	learn: 0.0517886	total: 1.04s	remaining: 636ms
620:	learn: 0.0517507	total: 1.04s	remaining: 634ms
621:	learn: 0.0517215	total: 1.04s	remaining: 632ms
622:	learn: 0.0517050	total: 1.04s	remaining: 631ms
623:	learn: 0.0516589	total: 1.04s	remaining: 629ms
624:	learn: 

838:	learn: 0.0440162	total: 1.39s	remaining: 268ms
839:	learn: 0.0439894	total: 1.4s	remaining: 266ms
840:	learn: 0.0439117	total: 1.4s	remaining: 264ms
841:	learn: 0.0438992	total: 1.4s	remaining: 263ms
842:	learn: 0.0438768	total: 1.4s	remaining: 261ms
843:	learn: 0.0437725	total: 1.4s	remaining: 259ms
844:	learn: 0.0436992	total: 1.4s	remaining: 258ms
845:	learn: 0.0436706	total: 1.41s	remaining: 256ms
846:	learn: 0.0436584	total: 1.41s	remaining: 254ms
847:	learn: 0.0436286	total: 1.41s	remaining: 253ms
848:	learn: 0.0436166	total: 1.41s	remaining: 251ms
849:	learn: 0.0435925	total: 1.41s	remaining: 249ms
850:	learn: 0.0435652	total: 1.41s	remaining: 248ms
851:	learn: 0.0435429	total: 1.42s	remaining: 246ms
852:	learn: 0.0435216	total: 1.42s	remaining: 244ms
853:	learn: 0.0435098	total: 1.42s	remaining: 243ms
854:	learn: 0.0434678	total: 1.42s	remaining: 241ms
855:	learn: 0.0434491	total: 1.42s	remaining: 239ms
856:	learn: 0.0434141	total: 1.42s	remaining: 238ms
857:	learn: 0.0433

...

Learning rate set to 0.040175
0:	learn: 0.1362235	total: 1.91ms	remaining: 1.91s
1:	learn: 0.1351227	total: 3.36ms	remaining: 1.68s
2:	learn: 0.1340048	total: 4.98ms	remaining: 1.65s
3:	learn: 0.1328176	total: 6.49ms	remaining: 1.62s
4:	learn: 0.1316543	total: 8.16ms	remaining: 1.62s
5:	learn: 0.1305370	total: 9.76ms	remaining: 1.62s
6:	learn: 0.1297514	total: 10.7ms	remaining: 1.51s
7:	learn: 0.1288139	total: 12.1ms	remaining: 1.5s
8:	learn: 0.1278963	total: 13.7ms	remaining: 1.51s
9:	learn: 0.1270442	total: 15.3ms	remaining: 1.51s
10:	learn: 0.1260740	total: 17ms	remaining: 1.53s
11:	learn: 0.1250523	total: 19ms	remaining: 1.57s
12:	learn: 0.1241410	total: 20.7ms	remaining: 1.57s
13:	learn: 0.1233430	total: 22.4ms	remaining: 1.58s
14:	learn: 0.1225178	total: 23.9ms	remaining: 1.57s
15:	learn: 0.1215157	total: 25.6ms	remaining: 1.57s
16:	learn: 0.1207085	total: 27.4ms	remaining: 1.58s
17:	learn: 0.1199416	total: 28.9ms	remaining: 1.58s
18:	learn: 0.1192626	total: 30.5ms	remaining: 1.5

236:	learn: 0.0762983	total: 382ms	remaining: 1.23s
237:	learn: 0.0761782	total: 384ms	remaining: 1.23s
238:	learn: 0.0760515	total: 385ms	remaining: 1.23s
239:	learn: 0.0759889	total: 387ms	remaining: 1.23s
240:	learn: 0.0758547	total: 389ms	remaining: 1.22s
241:	learn: 0.0757256	total: 391ms	remaining: 1.22s
242:	learn: 0.0756646	total: 392ms	remaining: 1.22s
243:	learn: 0.0755273	total: 394ms	remaining: 1.22s
244:	learn: 0.0754638	total: 396ms	remaining: 1.22s
245:	learn: 0.0753320	total: 398ms	remaining: 1.22s
246:	learn: 0.0752097	total: 400ms	remaining: 1.22s
247:	learn: 0.0751675	total: 402ms	remaining: 1.22s
248:	learn: 0.0750359	total: 403ms	remaining: 1.22s
249:	learn: 0.0749123	total: 405ms	remaining: 1.21s
250:	learn: 0.0747925	total: 406ms	remaining: 1.21s
251:	learn: 0.0746687	total: 408ms	remaining: 1.21s
252:	learn: 0.0746105	total: 409ms	remaining: 1.21s
253:	learn: 0.0745764	total: 411ms	remaining: 1.21s
254:	learn: 0.0745189	total: 412ms	remaining: 1.2s
255:	learn: 0

474:	learn: 0.0591458	total: 763ms	remaining: 843ms
475:	learn: 0.0590919	total: 765ms	remaining: 842ms
476:	learn: 0.0590580	total: 766ms	remaining: 840ms
477:	learn: 0.0589905	total: 768ms	remaining: 839ms
478:	learn: 0.0589334	total: 770ms	remaining: 837ms
479:	learn: 0.0588797	total: 771ms	remaining: 836ms
480:	learn: 0.0588521	total: 773ms	remaining: 834ms
481:	learn: 0.0587910	total: 775ms	remaining: 833ms
482:	learn: 0.0587266	total: 777ms	remaining: 831ms
483:	learn: 0.0586710	total: 779ms	remaining: 830ms
484:	learn: 0.0586164	total: 780ms	remaining: 829ms
485:	learn: 0.0585617	total: 783ms	remaining: 828ms
486:	learn: 0.0585153	total: 785ms	remaining: 827ms
487:	learn: 0.0584956	total: 787ms	remaining: 825ms
488:	learn: 0.0584188	total: 788ms	remaining: 824ms
489:	learn: 0.0583509	total: 790ms	remaining: 822ms
490:	learn: 0.0582863	total: 792ms	remaining: 821ms
491:	learn: 0.0582261	total: 794ms	remaining: 820ms
492:	learn: 0.0582006	total: 796ms	remaining: 818ms
493:	learn: 

699:	learn: 0.0491240	total: 1.15s	remaining: 491ms
700:	learn: 0.0491105	total: 1.15s	remaining: 490ms
701:	learn: 0.0490686	total: 1.15s	remaining: 488ms
702:	learn: 0.0490395	total: 1.15s	remaining: 486ms
703:	learn: 0.0490046	total: 1.15s	remaining: 485ms
704:	learn: 0.0489544	total: 1.16s	remaining: 484ms
705:	learn: 0.0489183	total: 1.16s	remaining: 482ms
706:	learn: 0.0488910	total: 1.16s	remaining: 480ms
707:	learn: 0.0488571	total: 1.16s	remaining: 479ms
708:	learn: 0.0488082	total: 1.16s	remaining: 477ms
709:	learn: 0.0487773	total: 1.16s	remaining: 476ms
710:	learn: 0.0487414	total: 1.17s	remaining: 474ms
711:	learn: 0.0486987	total: 1.17s	remaining: 472ms
712:	learn: 0.0486703	total: 1.17s	remaining: 471ms
713:	learn: 0.0486378	total: 1.17s	remaining: 469ms
714:	learn: 0.0486079	total: 1.17s	remaining: 468ms
715:	learn: 0.0485730	total: 1.17s	remaining: 466ms
716:	learn: 0.0485163	total: 1.18s	remaining: 464ms
717:	learn: 0.0484901	total: 1.18s	remaining: 463ms
718:	learn: 

930:	learn: 0.0414892	total: 1.53s	remaining: 113ms
931:	learn: 0.0414647	total: 1.53s	remaining: 112ms
932:	learn: 0.0414354	total: 1.53s	remaining: 110ms
933:	learn: 0.0413769	total: 1.53s	remaining: 108ms
934:	learn: 0.0413253	total: 1.53s	remaining: 107ms
935:	learn: 0.0413039	total: 1.54s	remaining: 105ms
936:	learn: 0.0412817	total: 1.54s	remaining: 103ms
937:	learn: 0.0412532	total: 1.54s	remaining: 102ms
938:	learn: 0.0412345	total: 1.54s	remaining: 100ms
939:	learn: 0.0412188	total: 1.54s	remaining: 98.6ms
940:	learn: 0.0411852	total: 1.55s	remaining: 97ms
941:	learn: 0.0411548	total: 1.55s	remaining: 95.4ms
942:	learn: 0.0411181	total: 1.55s	remaining: 93.8ms
943:	learn: 0.0410992	total: 1.55s	remaining: 92.1ms
944:	learn: 0.0410777	total: 1.55s	remaining: 90.5ms
945:	learn: 0.0410539	total: 1.56s	remaining: 88.9ms
946:	learn: 0.0410249	total: 1.56s	remaining: 87.2ms
947:	learn: 0.0410068	total: 1.56s	remaining: 85.6ms
948:	learn: 0.0409774	total: 1.56s	remaining: 83.9ms
949:

In [27]:
for result in results:
    print(f'{result["model_name"]}:\t{result["result"]}')

Standrd Linear Regression:	260701567128.86816
Decision Tree Regressor:	0.10268946140770178
Random Forest Regressor:	0.08088968715809991
LGBM Regressor:	0.10805722783032763
XGBoost Regressor:	0.06181914948398783
CatBoost Regressor:	0.06179722767610065
Stochastic Gradient Descent Regression:	0.12321701221861328
Kernel Ridge Regression:	0.06751095490277917
Elastic Net Regression:	0.13511346072353092
Bayesian Ridge Regression:	0.058045158922319894
Gradient Boosting Regression:	0.08496950807190017
Support Vector Machine:	0.0735063882359149


## Save Good Models

In [28]:
good_models = []

### Save Models

In [29]:
average_model_accuracy = 0
count = 0

for model in results:
    average_model_accuracy += model["result"]
    count += 1

average_model_accuracy /= count

for model in results:
    if model["result"] < average_model_accuracy:
        good_models.append(model["model"])

In [30]:
print(good_models)

[DecisionTreeRegressor(), RandomForestRegressor(), LGBMRegressor(), XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...), <catboost.core.CatBoostRegressor object at 0x000001C09CF41D80>, SGDRegressor(), KernelRidge(), ElasticNet(), BayesianRidge(), GradientBoostingRegressor(), SVR()]


# Happy Stocks

## Get Stock Info

In [31]:
blue_chip_stock_200DayMovingAverage_max = 300
api_keys = [
    "2RFQ9MPXDZ19H8ZB", "5HNZLHHXQ49419OW", "AD4QHR1QWWPX3PAZ",
    "DROMFL8PK13N9PVP", "0Z6NF5RSECY0KSUT", "43GLZYIC6DFBYGIT",
    "KUJBKMRA90WZSS7B"]

In [32]:
'''
Year: Founding year moduloed and adjusted
Country: Country
Happiness: Sentiment Analysis
Economy: Stock Price
Health: Profit Margin
Freedom: DividendYield
Corruption: MarketCapitalization

Generosity: How much to buy (or not)
'''

def transform_stock(symbol):
    column_names = train_set_x.columns.values
    column_values = train_set_x.columns.values
    
    # Get General Data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={random.choice(api_keys)}'
    r = requests.get(url)
    data = r.json()
    
    # Year
    column_values[0] = 1
 
    csv_file = csv.reader(open('pickable_stocks.csv', 'r'), delimiter=',')
    
    for row in csv_file:
        if symbol == row[7]:
            if row[7] != None:
                column_values[0] = ((row[7]%8)+2023)/8
            break
    
    # Country
    country = data["Country"]
    if country == "USA":
        country = 'United States'
    country = f"('{country}',)"
    
    for i in range(6, len(column_values)):
        if column_values[i] == country:
            column_values[i] = 1
        else:
            column_values[i] = 0
    
    # Economy
    column_values[2] = float(data["200DayMovingAverage"])/blue_chip_stock_200DayMovingAverage_max
    
    # Health
    column_values[3] = float(data['ProfitMargin'])
    
    # Freedom
    column_values[4] = min(float(data['DividendYield']) * 100, 1)
    
    # Corruption
    column_values[5] = min(float(data['MarketCapitalization']) / (2 * (10**12)), 1)
    
    # Get Sentiment Analysis Data
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers{symbol}&apikey={random.choice(api_keys)}'
    r = requests.get(url)
    data = r.json()
    
    # Happiness
    happiness = []
    for item in data["feed"]:
        for ticker in item["ticker_sentiment"]:
            if ticker["ticker"] == symbol:
                happiness.append(float(ticker["ticker_sentiment_score"]) * float(ticker["relevance_score"]))

    if len(happiness) == 0:
       column_values[1] = 0.5
    else:
        score = ((sum(happiness)/len(happiness)) + 0.35) / 0.7
        column_values[2] = max(min(score, 1), 0)
    
    return pd.DataFrame([column_values], columns=column_names)

In [33]:
def predict_stock(stock, models):
    stock_transform = transform_stock(stock)
    predictions = []
    
    for model in good_models:
        predictions.append(model.predict(stock_transform.values))
    
    prediction = sum(predictions)[0] / len(predictions)
    
    return prediction

In [40]:
def select_random_stock():
    csv_file = csv.reader(open('pickable_stocks.csv', 'r'), delimiter=',')
    chosen_row = random.choice(list(csv_file))
    
    print("Selected ", chosen_row[0])
    
    return chosen_row[0]

In [35]:
stocks = ["EBAY", "AAPL", "WMT", "AMZN"]

for stock in stocks:
    print(stock, " ", predict_stock(stock, good_models))
    for i in range(6):
        print(i)
        time.sleep(10)

EBAY   0.2344531046740378
0
1
2
3
4
5
AAPL   0.21974986683807743
0
1
2
3
4
5
WMT   0.23626325908469434
0
1
2
3
4
5
AMZN   0.16205376381258266
0
1
2
3
4
5


In [41]:
predict_stock(select_random_stock(), good_models)

Selected  CINF


0.23969893726506336